In [1]:
import os, json, pprint, re
from tqdm import tqdm

import pandas as pd

In [19]:
folder_no = 2
folder_path = f"C:/Users/Jin/PycharmProjects/KDT/json_file{folder_no}/"

In [9]:
def regular_expression(text):
    result = re.sub(r'o/|n/|b/ ', r'', text)
    result = re.sub(r'\(\(\)\)',r'', result)
    result = re.sub(r'/\(([^)]+)\)', r'', result)
    result = re.sub(r'\(', r'', result, count=10)
    result = re.sub(r'\)', r'', result, count=10)
    result = re.sub(r'/', r'', result, count=10).strip()
    return "".join(result)

In [20]:
from collections import defaultdict
def folder_file_to_df(folder_path):
    
    target_folder = os.listdir(folder_path)
    print(folder_path)
    
    cnt = 0
    dialog_dict = defaultdict(list)
    df = pd.DataFrame(columns=['speaker_type', 'speaker', 'txt'])
    txt_list = []
    
    # 내 코드는 왜 항상 느림
    # for file in tqdm(target_folder):
    #     file_path = os.path.join(folder_path, file)
    #     try:
    #         target_file = open(file_path, encoding="utf-8")
    #         data = json.load(target_file)
    #         category = data['dataSet']['typeInfo']['subcategory']
    #         if category not in ('복지', '일반행정', '기타민원'):
    #             continue
    #         
    #         speakers = data['dataSet']['typeInfo']['speakers']
    #         speaker = None
    #     
    #     
    #         for text in data['dataSet']['dialogs']:
    #             speaker = text['speaker']
    #             speaker_type = speakers[int(speaker)-1]['type']
    #             txt = regular_expression(text['text'])
    #             temp_df = pd.DataFrame({
    #                 'speaker_type': speaker_type, 
    #                 'speaker': speaker, 
    #                 'txt': txt, 
    #                 'label':category}, 
    #                 index=[0])
    #             df = pd.concat([df, temp_df], axis=0, ignore_index=True)
    #     except:
    #         print(f"error : {file_path}")
    
    # GPT가 바꿔준 코드 개빠름 ㅠ
    data_list = []
    for file in tqdm(target_folder):
        file_path = os.path.join(folder_path, file)
        try:
            with open(file_path, encoding="utf-8") as target_file:
                data = json.load(target_file)
                category = data['dataSet']['typeInfo']['subcategory']
                if category not in ('복지', '일반행정', '기타민원'):
                    continue
    
                speakers = data['dataSet']['typeInfo']['speakers']
    
                for text in data['dataSet']['dialogs']:
                    speaker = text['speaker']
                    speaker_type = None
                    for s in speakers:
                        if s['id'] == speaker:
                            speaker_type = s['type']
                    txt = regular_expression(text['text'])
                    
                    # 데이터 리스트에 추가
                    data_list.append({
                        'speaker_type': speaker_type, 
                        'speaker': speaker, 
                        'txt': txt, 
                        'label': category
                    })
        except Exception as e:
            print(f"error : {file_path}, {str(e)}")
    
    return pd.DataFrame(data_list)

# 한 번에 DataFrame으로 변환
df = folder_file_to_df(folder_path)

C:/Users/Jin/PycharmProjects/KDT/json_file2/


100%|██████████| 9481/9481 [00:03<00:00, 3027.23it/s]


In [24]:
# 파일 만들어줌
print(df['label'].value_counts())
df.to_csv(f'data_file/test{folder_no}.csv', encoding='utf-8', index=False)

label
일반행정    180454
Name: count, dtype: int64


In [26]:
test_file_path = f"data_file/test{folder_no}.csv"

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

{'x': {'type': '고객', 'id': {'2'}, 'text': []}, 'y': {'type': '상담사', 'id': {'3', '1'}, 'text': []}}


In [27]:
test_df = pd.read_csv(f"data_file/test{folder_no}.csv")
test_df.dropna(inplace=True)

In [28]:
print(test_df.info())



<class 'pandas.core.frame.DataFrame'>
Index: 180304 entries, 0 to 180453
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speaker_type  180304 non-null  object
 1   speaker       180304 non-null  int64 
 2   txt           180304 non-null  object
 3   label         180304 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.9+ MB
None


In [15]:
# # 고객 X, 상담사 Y로 나누고 분석하기 편하게 바꿈
# target_file = open(test_file_path, encoding="UTF-8") # json file load
# json_file = json.loads(target_file.read())
# speakers = json_file['dataSet']['typeInfo']['speakers']
# category = json_file['dataSet']['typeInfo']['subcategory']
# s = []
# for speaker in speakers:
#     s.append((speaker['type'], speaker['id']))
# print(s)
# 
# print(category)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
def merge_and_transform(df):
    customer_txt = []
    counselor_txt = []

    current_speaker_type = df.iloc[0]['speaker_type'][:-1]  # 고객1 -> 고객
    current_txt = []

    for i, row in df.iterrows():
        normalized_speaker_type = row['speaker_type'][:-1]  # speaker_type 정규화
        
        if normalized_speaker_type == current_speaker_type:
            current_txt.append(row['txt'])
        else:
            if current_speaker_type == '고객':
                customer_txt.append(" ".join(current_txt))
            else:
                counselor_txt.append(" ".join(current_txt))

            current_speaker_type = normalized_speaker_type
            current_txt = [row['txt']]
    
    # 마지막 대화 누적
    if current_speaker_type == '고객':
        customer_txt.append(" ".join(current_txt))
    else:
        counselor_txt.append(" ".join(current_txt))

    # 고객 대화가 더 많다면 마지막 항목을 제거
    if len(customer_txt) > len(counselor_txt):
        customer_txt = customer_txt[:-1]

    # X(고객 대화), Y(상담사 대화), label로 새로운 DataFrame 생성
    final_df = pd.DataFrame({
        'X': customer_txt,
        'Y': counselor_txt,
        'label': df['label'].iloc[0]  # 모든 대화가 같은 label을 갖는다고 가정
    })

    return final_df

In [30]:
# Process the dataframe
merged_df = merge_and_transform(test_df)


In [31]:
print(merged_df.shape)
merged_df.to_csv(f"data_file/merged_df{folder_no}.csv", encoding='utf-8', index=False)

(41218, 3)


In [99]:
merged_df.isna().sum()

X        0
Y        0
label    0
dtype: int64

In [100]:
merged_df

,X,Y,label
0,"네, 안녕하세요?","네, 안녕하십니까?",복지
1,다름이 아니고 저희 아들이 디딤 씨앗 통장을. 해지하려고 하는데요.,죄송합니다만 제가 답변이 어렵기 때문에 담당자 연결해 드리도록 하겠습니다.,복지
2,"네, 알겠습니다.","고객님 혹시라도 연결 중 전화가 끊긴다면. 예, 담당자 바로 연결해드리겠습니다.",복지
3,"네, 알겠습니다.","네, 좋은 상담 이어 가세요. 고객님께서 디딤 씨앗 통장 해지 관련으로 문의하세요....",복지
4,아 저 몇 가지 여쭤볼 게 있어가지고요. 여쭤보려면 어디로 통화를 해야 되나요?,혹시 어떤 불편사항이신가요? 고객님.,복지
...,...,...,...
65845,아니 그거 저기 뭐꼬 희망 근로나 저 저 뭐 공공 근로 이런 거 하면은 이게 아는 ...,제가 담당자가 아니고 저희도 콜센턴데요. 담당자를 직접 연결해,복지
65846,콜 아까도 그러면 콜센터에서 전화를 받으셨습니까? 여자분 받을 때도 콜센터입니까?,아까도 콜센터에서 받으셨는지는 제가 지금 확인은 어렵고 전화번호를 말씀해 주셔야지 ...,복지
65847,그래 제가 1 마디만 더 드릴 테니까 그거는. 사람이 일을 오래 하니까 뒤에 들어간...,지금 먼저 접수해 주신 내용이 있다면 담당자에게 직접 전달될 수 있도록 연결해 봐 ...,복지
65848,아 그거는 고마 그러면 담당자를 또 통화하면은 고마 제가 지금 여기 치과에 왔거든요...,전달해 드리겠습니다.,복지
